In [1]:
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup

import time

In [2]:
names = ['Manchester United', 'Tottenham', 'Bournemouth', 'Aston Villa',
        'Everton', 'Watford', 'Leicester', 'Sunderland', 'Norwich',
        'Crystal Palace', 'Chelsea', 'Swansea', 'Newcastle United',
        'Southampton', 'Arsenal', 'West Ham', 'Stoke', 'Liverpool',
        'West Bromwich Albion', 'Manchester City', 'Hull', 'Burnley',
        'Middlesbrough', 'Queens Park Rangers', 'Huddersfield', 'Brighton',
        'Fulham', 'Cardiff', 'Wolverhampton Wanderers', 'Sheffield United',
        'Leeds']

In [3]:
link = ['https://sofifa.com/teams?type=club&ct%5B0%5D=2&na%5B0%5D=14&r=160058&set=true',
       'https://sofifa.com/teams?type=club&ct%5B0%5D=2&na%5B0%5D=14&r=170099&set=true',
       'https://sofifa.com/teams?type=club&ct%5B0%5D=2&na%5B0%5D=14&r=180084&set=true',
       'https://sofifa.com/teams?type=club&ct%5B0%5D=2&na%5B0%5D=14&r=190075&set=true',
       'https://sofifa.com/teams?type=club&ct%5B0%5D=2&na%5B0%5D=14&r=200061&set=true',
       'https://sofifa.com/teams?type=club&ct%5B0%5D=2&na%5B0%5D=14&r=210064&set=true']

for url in link:
    
    response = requests.get(url)
    if response.status_code == 200:
        html = response.content
        soup = BeautifulSoup(html, 'lxml')
        
        
        team_names = soup.select('div[class="bp3-text-overflow-ellipsis"]')
        team_transfer_budget = soup.select('td[class="col col-tb"]')
        team_overall = soup.select('td[class="col col-oa"] > span')

        team = pd.DataFrame(columns=['name', 'transfer_budget', 'overall'], index=range(0, 20))
        count = 0
        for i in range(len(team_overall)):
            if team_names[2*i+1].text.strip() == 'English Premier League (1)':
                team.iloc[count, 0] = team_names[2*i].text.strip()
                team.iloc[count, 2] = team_overall[i].text.strip()
        
                budget = team_transfer_budget[i].text.strip()
                if budget[-1] == 'M':
                    team.iloc[count, 1] = float(budget[1:-1])*1000000
                    count += 1
                elif budget[-1] == 'K':
                    team.iloc[count, 1] = float(budget[1:-1])*1000
                    count += 1
        
        for i in range(len(team['name'])):
            for j in names:
                if j in team['name'][i]:
                    team.iloc[i, 0] = j
            
    team.to_csv(f'data\\scrapped\\fifa_teams_{url[-15:-13]}.csv')
    time.sleep(5)